In [30]:
#imports

import cv2
from cv2 import threshold
import numpy as np
import tracker.tracker as tracker
import identifier.identifier as identifier
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# initial setup

threshold = 20
webcam = cv2.VideoCapture(0)
ret, frame = webcam.read()

# Will's webcam dims are: 480, 640, 3
height, width, d = frame.shape
print(height, width)

#this should turn drawing area into square
line_x = width - height
box_width = 50 #arbitrarily set


# this is used to creat the test data
# set to false when not recording webcam
recording = False
if recording:
    out = cv2.VideoWriter("Test_video2.avi", cv2.VideoWriter_fourcc(*'MJPG'), 10,(width, height)) 

xcenter_bw = 240
ycenter_bw = 240

xcenter = 400
ycenter = 240

sync_finger = None
sync_finger_bw = None
sync_frame = None
next_frame = None
while(True):
		
    # Capture the video frame
    # by frame
    if next_frame is None:
        ret, frame = webcam.read()
        _, next_frame = webcam.read()
    else:
        frame = next_frame
        _, next_frame = webcam.read()


    # flip so it makes sense to the user for drawing
    frame = cv2.flip(frame, 1)
    if recording:
        out.write(frame)


    # duplicate current frame
    # one is used for processing the other is show to user
    c_frame = frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # sync with background by pressing 's'
    if cv2.waitKey(1) & 0xFF == ord('s'):
        sync_frame = frame[:, line_x:]


    if sync_frame is not None:

        #  remove background from current frame
        cut_frame =  frame[:, line_x:]
        cut_frame = cv2.subtract(sync_frame, cut_frame)


        # create binary image
        im_bw = cv2.threshold(cut_frame, threshold, 255, cv2.THRESH_BINARY)[1]
        coords = identifier.get_fingertip(im_bw)

        fingers = len(coords)

        for coord in coords:
            x, y = coord

            cv2.circle(im_bw, (x, y), 10, (255, 255, 255), 2)

            cv2.circle(c_frame, (x + line_x, y), 10, (255, 0, 0), 2)

        cv2.imshow('bin', im_bw)


    # Display the resulting frame
    # this will be what the user ends up seeing
    cv2.line(c_frame, (line_x, -10), (line_x, height + 10), (0, 255, 0), thickness=3)
    

    cv2.imshow('frame', c_frame)




# After the loop release the cap object
webcam.release()
if recording:
    out.release()

# Destroy all the windows
cv2.destroyAllWindows()



480 640
All: [(195, 408)]
All: [(192, 346), (332, 416)]
All: [(188, 343), (329, 417)]
All: [(180, 340), (320, 416)]
All: [(177, 337), (317, 423)]
All: [(255, 393)]
All: [(255, 393)]
All: [(214, 390), (354, 466)]
All: [(168, 249)]
All: [(161, 204)]
All: [(158, 167)]
All: [(158, 167)]
All: [(146, 150)]
All: [(143, 134)]
All: [(133, 126)]
All: [(124, 119)]
All: [(115, 115)]
All: [(113, 114)]
All: [(110, 112)]
All: [(109, 111)]
All: [(107, 108)]
All: [(111, 110)]
All: [(114, 111)]
All: [(120, 113)]
All: [(123, 116)]
All: [(129, 119)]
All: [(136, 123)]
All: [(138, 126)]
All: [(141, 129)]
All: [(143, 132)]
All: [(145, 137)]
All: [(144, 140)]
All: [(144, 142)]
All: [(147, 145)]
All: [(142, 148)]
All: [(140, 151)]
All: [(144, 154)]
All: [(133, 157)]
All: [(124, 157)]
All: [(117, 161), (311, 271)]
All: [(108, 160), (317, 205)]
All: [(114, 157), (315, 186)]
All: [(117, 153), (317, 179)]
All: [(118, 154), (321, 178)]
All: [(116, 151), (327, 174)]
All: [(115, 151), (319, 173)]
All: [(119, 148), (3